## Imports

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from hbt.utils_hbt import jsm, gm
from pathlib import Path

## Settings

In [14]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

## Functions

In [2]:
def split_df_year(df):
    dfs = {}
    years = df.index.year.unique()
    for year in years:
        df_tmp = df[df.index.year == year]
        df_tmp.index = df_tmp.index.month.unique()
        dfs.update({year: df_tmp})
    return dfs 

## Setup

In [3]:
# Paths
path_ara = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung"
    r"\01_Rohdaten\Zulaufmenge_ARA_Skript_1min.csv")
path_rain = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung"
    r"\01_Rohdaten\data_1d.csv")
path_twv_m = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung\01_Rohdaten"
    r"\Trinkwasserverbrauch_Monat.csv")
path_twv_y = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung"
    r"\02_Bereinigte_Daten\Trinkwasserverbrauch\Trinkwasserverbrauch.csv")
path_ew_y = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung"
    r"\02_Bereinigte_Daten\Einwohner\Einwohner.csv")
path_jsm_m_tw = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung\03_Auswertung"
    r"\Jahresschmutzwasser\Monatswerte_Zusammensetzung_tw.csv")
path_jsm_m_ewg = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung\03_Auswertung"
    r"\Jahresschmutzwasser\Monatswerte_Zusammensetzung_ewg.csv")
path_nm_m_03 = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung\03_Auswertung"
    r"\Nachtminimum\Monatswerte_Zusammensetzung_03.csv")
path_nm_m_05 = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung\03_Auswertung"
    r"\Nachtminimum\Monatswerte_Zusammensetzung_05.csv")
path_gm_m_tw = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung\03_Auswertung"
    r"\GleitendesMinimum\Monatswerte_Zusammensetzung_tw.csv")
path_gm_m_ewg = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Auswertung\03_Auswertung"
    r"\GleitendesMinimum\Monatswerte_Zusammensetzung_ewg.csv")

save_dir = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\ARA\Plots")

## Read data

In [4]:
# Read ARA data
date_format = '%d.%m.%Y %H:%M:%S'
date_parser = lambda x: pd.to_datetime(x, dayfirst=True,
    format=date_format)
ts_ara_1min = pd.read_csv(path_ara, header=None, skiprows=1, parse_dates=True,
    date_parser=date_parser, dayfirst=True, index_col=0, sep=';')

# Volume per day (m3)
ts_ara_d = ts_ara_1min.groupby(pd.Grouper(freq='D')).sum() * (60/1e3)

In [5]:
# Read rain data
date_format = '%d.%m.%Y'
date_parser = lambda x: pd.to_datetime(x, dayfirst=True,
    format=date_format)
df_rain_d = pd.read_csv(path_rain, header=None, skiprows=1, parse_dates=True,
    date_parser=date_parser, dayfirst=True, index_col=0, sep=';')

In [6]:
# Read Trinwasserverbrauch data
date_format = '%m.%Y'
date_parser = lambda x: pd.to_datetime(x, dayfirst=True,
    format=date_format)
df_twv_m = pd.read_csv(path_twv_m, header=0, parse_dates=True,
    date_parser=date_parser, dayfirst=True, index_col=0, sep=';')

date_format = '%Y'
date_parser = lambda x: pd.to_datetime(x, dayfirst=True,
    format=date_format)
ts_twv_y = pd.read_csv(path_twv_y, header=None, skiprows=1, parse_dates=True,
    date_parser=date_parser, index_col=0, sep=';')
ts_ew_y = pd.read_csv(path_ew_y, header=None, skiprows=1, parse_dates=True,
    date_parser=date_parser, index_col=0, sep=';')

## Plots

### ARA Zufluss

In [7]:
# Compute hourly-mean
df_ara_1h_mean = ts_ara_1min.groupby(pd.Grouper(freq='H')).mean()

# Extract daily-mins
df_ara_d_min = df_ara_1h_mean.groupby(pd.Grouper(freq='D')).min()

# Plot the data
fig, axs = plt.subplots()
axs.plot(df_ara_d_min)
axs.set_xlabel('Datum')
axs.set_ylabel('Zufluss [l/s]')
axs.set_title('Tagesminimum')

fig.set_size_inches(15,10)
fig.set_facecolor('white')
fig.set_dpi(300)

save_path = save_dir / 'Tagesminimum.png'
fig.savefig(save_path)
plt.close()

### Histogramm

In [28]:
data = ts_ara_1min
data = data[data<=150]
n_bins = int(data.max() - data.min())

cm_in = 0.3937
w=14.5*cm_in
h=10*cm_in
fig, ax = plt.subplots(figsize=(w, h), facecolor='white')
plt.autoscale(enable=True, axis='x', tight=True)

txt_props = {
    'font': 'Arial',
    'fontsize': 9
}
n, bins, patches = ax.hist(data, bins=n_bins, edgecolor='k', linewidth=0.5)
ax.set_xlabel('Durchfluss [l/s]', txt_props)
ax.set_ylabel('Anzahl', txt_props)

ax.tick_params(labelsize=8)

y_lims = ax.get_ybound()
ax.axvline(x=7, color='r', linewidth=0.5)
ax.axvline(x=21, c='r', linewidth=0.5)

fig.set_dpi(300)

save_path = save_dir / 'Histogramm.png'
fig.savefig(save_path, bbox_inches='tight')
plt.close()

### Jahresschmutzwasser / Nachtsminimum / Gleitendes Minimum

In [31]:
# Jahresschmutzwasser
## Read data
date_format = '%m.%Y'
date_parser = lambda x: pd.to_datetime(x, dayfirst=True,
    format=date_format)

### TW
jsm_m_tw = pd.read_csv(path_jsm_m_tw, header=None, skiprows=0,
    parse_dates=True, date_parser=date_parser, dayfirst=True, index_col=0,
    sep=';')

### Convert to l/s
c_fact = 1e3 / (24*60**2)
jsm_m_tw = jsm_m_tw * c_fact

### Split in years
jsm_m_tw = split_df_year(jsm_m_tw)

### Plot data
fig, axs = plt.subplots(3)
fig.set_size_inches(10,16)
fig.set_facecolor('white')
fig.set_dpi(300)

leg = []
col = ['r', 'b', 'g']

#### TW
for i, (y, d) in enumerate(jsm_m_tw.items()):
    axs[0].plot(d, col[i])
    leg.append(('TW ' + str(y)))

axs[0].legend(leg)
axs[0].set_xlabel('Monat')
axs[0].set_ylabel('FW [l/s]')
axs[0].set_title('Jahresschmutzwasser')
#-------------------------------------------------------------------------------

# Nachtsminimum
## Read data
### 0.3
nm_m_03 = pd.read_csv(path_nm_m_03, header=None, skiprows=0,
    parse_dates=True, date_parser=date_parser, dayfirst=True, index_col=0,
    sep=';')

### 0.5
nm_m_05 = pd.read_csv(path_nm_m_05, header=None, skiprows=0,
    parse_dates=True, date_parser=date_parser, dayfirst=True, index_col=0,
    sep=';')

## Split in years
nm_m_03 = split_df_year(nm_m_03)
nm_m_05 = split_df_year(nm_m_05)

## Plot data
leg = []

### 0.3
for i, (y, d) in enumerate(nm_m_03.items()):
    axs[1].plot(d, col[i])
    leg.append((str(y) + ' (GW = 0.3mm)'))

### 0.5
for i, (y, d) in enumerate(nm_m_05.items()):
    axs[1].plot(d, col[i], linestyle='--')
    leg.append((str(y) + ' (GW = 0.5mm)'))

axs[1].legend(leg)
axs[1].set_xlabel('Monat')
axs[1].set_ylabel('FW [l/s]')
axs[1].set_title('Nachtsminimum')
#-------------------------------------------------------------------------------

# Gleitendes Minimum
## Read data
### TW
gm_m_tw = pd.read_csv(path_gm_m_tw, header=None, skiprows=0,
    parse_dates=True, date_parser=date_parser, dayfirst=True, index_col=0,
    sep=';')

### Convert to l/s
gm_m_tw = gm_m_tw * c_fact

## Split in years
gm_m_tw = split_df_year(gm_m_tw)

## Plot data
### TW
leg = []
for i, (y, d) in enumerate(gm_m_tw.items()):
    axs[2].plot(d, col[i])
    leg.append(('TW ' + str(y)))

axs[2].legend(leg)
axs[2].set_xlabel('Monat')
axs[2].set_ylabel('FW [l/s]')
axs[2].set_title('Gleitendes Minimum')
save_path = save_dir / 'Monatswerte_Vergleich.png'
fig.savefig(save_path)
plt.close()


## Results

### JSM

In [ ]:
jsm_y_ewg_p = jsm(ts_ara_d=ts_ara_d, ts_rain_d=df_rain_d, ts_twv=ts_twv_y,
    freq_twv='year')
jsm_y_ewg_p = jsm_y_ewg_p * (1e3/(24*60**2))
print(jsm_y_ewg_p)

### GM

In [ ]:
# EW
fw_gm_ew_y, fw_gm_ew_m = gm(ts_ara_d=ts_ara_d, ts_ew=ts_ew_y)
print(fw_gm_ew_y)

# # TWV
# fw_gm_twv_y, fw_gm_twv_m = gm(ts_ara_d=ts_ara_d, ts_twv=ts_twv_y)
# print(fw_gm_twv_y)

### Yearly averages from months results

In [ ]:
# Compute yearly average
## JSM
def y_avg(m_dict):
    y_avg = dict()
    for y, df in m_dict.items():
        y_avg.update({y: df.mean().values[0]})
    return y_avg

def print_y_avg(y_avg, name):
    print(name)
    for y, val in y_avg.items():
        print(f"{y}:\t{val:0.1f}")
    print("")

jsm_y_ewg = y_avg(jsm_m_ewg)
print_y_avg(jsm_y_ewg, 'JSM EWG')
